# Quantified Self
By: Gavin McClure-Coleman & Sam Allen

## Introduction

For our project, we have decided to explore our Steam account data. Steam is a a digital distribution platform developed by Valve, primarily used for buying, downloading, and playing PC video games.

Using the Steam Web API through https://steamcommunity.com/dev and more specifically https://developer.valvesoftware.com/wiki/Steam_Web_API we are able to obtain all kinds of interesting stats like the games we own, have played the most, genres of those games, and much more.

We then decided to pull information about those games from the Steam Store API to compare our own data to some of the public information.

## 1) Get and Clean the Data

First, we need to make some API requests to get our data, we will be cleaning as we obtain this data to prep for our later exploratory data analysis.

To start, we will make a request to get all games owned on Gavin's Steam account

In [6]:
import utils
import pandas as pd

api_key = "D1E3D07E8AF97E4729A368F71FFB43F8"
sam_api_key = "BA9616D65081245B52940CEE86D4DECE"
steam_id = "76561198111062824"
sam_steam_id = "76561198198119993"

sam_games_df = utils.get_user_games(sam_api_key, sam_steam_id, "Sam")
gavin_games_df = utils.get_user_games(api_key, steam_id, "Gavin")

# Merge the two dataframes on the appid
if not sam_games_df.empty and not gavin_games_df.empty:
    # Make sure appid is a column in both dataframes
    merged_df = pd.merge(sam_games_df, gavin_games_df, on="appid", how="outer", suffixes=('_Sam', '_Gavin'))
    
    # For game names, use one version
    if 'name_Sam' in merged_df.columns and 'name_Gavin' in merged_df.columns:
        merged_df['Game'] = merged_df['name_Sam'].combine_first(merged_df['name_Gavin'])
        merged_df.drop(['name_Sam', 'name_Gavin'], axis=1, inplace=True)
    
    # Set the index to game name
    merged_df.set_index("Game", inplace=True)
    
    # Sort by Gavin's playtime
    if 'playtime_forever_Sam' in merged_df.columns:
        merged_df.sort_values(by="playtime_forever_Gavin", ascending=False, inplace=True)
    
    # Fill NaN values with 0 for playtime columns
    for col in merged_df.columns:
        if 'playtime' in col or 'rtime' in col:
            merged_df[col] = merged_df[col].fillna(0)
    
    df = merged_df
else:
    if not sam_games_df.empty:
        df = sam_games_df
    elif not gavin_games_df.empty:
        df = gavin_games_df
    else:
        df = pd.DataFrame()
        print("Could not retrieve games data for either user.")

df.head()


,appid,playtime_forever_Sam,img_icon_url_Sam,has_community_visible_stats_Sam,playtime_windows_forever_Sam,playtime_mac_forever_Sam,playtime_linux_forever_Sam,playtime_deck_forever_Sam,rtime_last_played_Sam,content_descriptorids_Sam,...,has_community_visible_stats_Gavin,playtime_windows_forever_Gavin,playtime_mac_forever_Gavin,playtime_linux_forever_Gavin,playtime_deck_forever_Gavin,rtime_last_played_Gavin,content_descriptorids_Gavin,playtime_disconnected_Gavin,has_leaderboards_Gavin,playtime_2weeks_Gavin
Game,,,,,,,,,,,,,,,,,,,,,
Rust,252490,56285.0,820be4782639f9c4b64fa3ca7e6c26a95ae4fd1c,True,56285.0,0.0,0.0,0.0,1.729369e+09,"[1, 2, 5]",...,True,64159.0,0.0,0.0,0.0,1.736583e+09,"[1, 2, 5]",0.0,NaN,0.0
Destiny 2,1085660,6822.0,fce050d63f0a2f8eb51b603c7f30bfca2a301549,True,6822.0,0.0,0.0,0.0,1.728678e+09,NaN,...,True,148733.0,0.0,0.0,0.0,1.745380e+09,NaN,0.0,NaN,453.0
Terraria,105600,57014.0,858961e95fbf869f136e1770d586e0caefd4cfac,True,51545.0,0.0,0.0,0.0,1.739931e+09,NaN,...,True,16965.0,0.0,0.0,0.0,1.745117e+09,NaN,0.0,NaN,5.0
Counter-Strike 2,730,4632.0,8dbc71957312bbd3baea65848b545be9eae2a355,True,1163.0,0.0,0.0,0.0,1.730015e+09,"[2, 5]",...,True,10.0,0.0,0.0,0.0,1.644630e+09,"[2, 5]",0.0,NaN,0.0
Tom Clancy's Rainbow Six Siege,359550,58551.0,624745d333ac54aedb1ee911013e2edb7722550e,NaN,29019.0,0.0,0.0,0.0,1.712614e+09,NaN,...,NaN,6793.0,0.0,0.0,0.0,1.744437e+09,NaN,0.0,NaN,46.0


## Data Cleaning
From the first 5 instances of our first dataset, we can see a couple of columns that aren't important to keep.  
Specifically, we will be getting rid of the following columns:
 * img_icon_url
 * has_community_visible_stats
 * playtime_windows_forever
 * playtime_mac_forever
 * playtime_linux_forever
 * playtime_deck_forever
 * content_descriptorids (describes the maturity ratings of the game)
 * playtime_disconnected
 * has_leaderboards

For clarity, the column rtime_last_played represents the **Unix timestamp** of the last time the game was played. In other words, it is the number of seconds since January 1, 1970 (the Unix epoch)
 * To convert from unix to a more readable format, we can use the `datetime.fromtimestamp(timestamp)` using the `from datetime import datetime` library.


In [7]:
df = utils.drop_columns(df, "Sam")
df = utils.drop_columns(df, "Gavin")
df.shape

(485, 7)

In [8]:
df.head()

,appid,playtime_forever_Sam,rtime_last_played_Sam,playtime_2weeks_Sam,playtime_forever_Gavin,rtime_last_played_Gavin,playtime_2weeks_Gavin
Game,,,,,,,
Rust,252490,56285.0,1.729369e+09,0.0,230031.0,1.736583e+09,0.0
Destiny 2,1085660,6822.0,1.728678e+09,0.0,148802.0,1.745380e+09,453.0
Terraria,105600,57014.0,1.739931e+09,0.0,73027.0,1.745117e+09,5.0
Counter-Strike 2,730,4632.0,1.730015e+09,0.0,42660.0,1.644630e+09,0.0
Tom Clancy's Rainbow Six Siege,359550,58551.0,1.712614e+09,0.0,39962.0,1.744437e+09,46.0


# Maybe add something here about how we want more data so we will use two more endpoints to get sale price, genres, review score, and percent of positive reviews

In [ ]:
import requests
import time

# Initialize lists to store data
genres = []
prices = []
review_descriptions = []
positive_ratios = []
negative_ratios = []


# Only use the first 5 games for testing
df = df.head(5)

def get_store_data(appid):
    store_url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=us&l=en"
    try:
        data = utils.make_request(store_url)
        if data[str(appid)]["success"]:
            return data[str(appid)]["data"]
        else:
            return None
    except Exception as e:
        print(f"Error getting store data for appid {appid}: {e}")
        return None

def get_review_data(appid):
    review_url = f"https://store.steampowered.com/appreviews/{appid}?json=1&language=all"
    try: 
        data = utils.make_request(review_url)
        if "query_summary" in data:
            summary = data['query_summary']
            desc = summary.get("review_score_desc")
            positive = summary.get("total_positive")
            negative = summary.get("total_negative")
            return desc, positive, negative
    except Exception as e:
        print(f"Error getting review data for appid {appid}: {e}")
        return None, None

# Loop through each game in our dataframe
for appid in df["appid"]:
    game_data = get_store_data(appid)
    review_desc, positive_ratio, negative_ratio = get_review_data(appid)
    time.sleep(0.2)  # We will be a little bit kind to steam and rate limit ourselves 😺

    if game_data:
        # Get genre(s)
        genre_list = game_data.get("genres")
        genre_names = [g["description"] for g in genre_list]
        genres.append(", ".join(genre_names) if genre_names else None)

        # Get price
        try:
            price_info = game_data["price_overview"]    
            prices.append(price_info["final"] / 100)  # Prices in dollars
        except KeyError:
            prices.append(0.0)
    else:
        genres.append(None)
        prices.append(None)
    
    review_descriptions.append(review_desc)
    positive_ratios.append(positive_ratio)
    negative_ratios.append(negative_ratio)

# Now update dataframe with our new data
df["Genre"] = genres
df["Price (USD $)"] = prices
df["Review Score"] = review_descriptions
df["Total Positive Reviews"] = positive_ratios
df["Total Negative Reviews"] = negative_ratios

# Display the result
df.head()


,appid,playtime_forever_Sam,rtime_last_played_Sam,playtime_2weeks_Sam,playtime_forever_Gavin,rtime_last_played_Gavin,playtime_2weeks_Gavin,Genre,Price (USD $),Review Score,Total Positive Reviews,Total Negative Reviews
Game,,,,,,,,,,,,
Rust,252490,56285.0,1.729369e+09,0.0,230031.0,1.736583e+09,0.0,"Action, Adventure, Indie, Massively Multiplaye...",39.99,Very Positive,883924,124266
Destiny 2,1085660,6822.0,1.728678e+09,0.0,148802.0,1.745380e+09,453.0,"Action, Adventure, Free To Play",0.00,Mostly Positive,100177,27449
Terraria,105600,57014.0,1.739931e+09,0.0,73027.0,1.745117e+09,5.0,"Action, Adventure, Indie, RPG",9.99,Overwhelmingly Positive,1087225,26208
Counter-Strike 2,730,4632.0,1.730015e+09,0.0,42660.0,1.644630e+09,0.0,"Action, Free To Play",0.00,Very Positive,3859823,614332
Tom Clancy's Rainbow Six Siege,359550,58551.0,1.712614e+09,0.0,39962.0,1.744437e+09,46.0,Action,7.99,Very Positive,997076,183817
